In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import tools
from importlib import reload


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

pd.options.mode.chained_assignment = None  # default='warn'

#### Ronda y fecha de cada ronda
fecha_round_1_matching = '2022_08_11'
fecha_round_2_matching = '2022_08_11'

#### Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)
#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"
path_dropbox = r"C:\Users\oficina825\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas Luis
# path_dropbox = r"C:\Users\37615965\Dropbox"

#### Rutas utilizadas (autogeneradas, no tocar):
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_datain = path_dropbox + r"\BID-LGBTQ+\campo\data\para_analisis"
path_dataout = path_dropbox + r"\BID-LGBTQ+\campo\data\mails_matches\matched\correccion_manual_de_matches"

#### Clave de ids
identifica_respuestas  = ['thread_id_inmob','fecha','from_inmob']
identifica_publicacion = ['link','municipio','id','ronda']

In [3]:
##################### IMPORTO BASES

#### Levanto la base "original-Full"
print('df_main')
df_main = pd.read_excel(path_datain + fr'\base_limpia_tarea_2_nico_sin_RA.xlsx')
df_main = tools.limpia_prueba_google_y_quien_sos(df_main, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')
df_main['match_manual'] = 0

#### Devoluciones de los RA + generación de id_respuesta
reload(tools)
print('\ndf')
df = tools.importa_y_genera_ids(
            path_dataout + '\RA_devolucion\Ronda-1', 
            'xlsx', 
            'id_respuesta', 
            identifica_respuestas
     )
df = tools.limpia_prueba_google_y_quien_sos(df, path_dropbox, from_name='from_inmob', subject_name='subject_inmob')
df.sort_values(by='fecha', inplace=True)
df.rename(columns={'Unnamed: 0':'id_merge'}, inplace=True)

#### Publicaciones enviadas a los RA +  generación de id_publicación
print('\npublicaciones_ra')
publicaciones_ra = tools.importa_y_genera_ids(
            path_dataout + r'\pubs_RA\ronda 1 - not matched', 
            'xlsx', 
            'id_publicacion_hash', 
            identifica_publicacion
     )

df_main
Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 63 observaciones de MDS.
Se dropearon 116 de ¿Quien sos?

df
Hay 10 archivos con la extension establecida dentro de la carpeta.
La base tiene 1215 observaciones.
Se dropearon 0 observaciones de Google.
Se dropearon 0 observaciones ('Julian Test').
Se dropearon 0 observaciones 'de prueba'.
Se dropearon 0 observaciones de mails nuestros.
Se dropearon 64 observaciones de MDS.
Se dropearon 101 de ¿Quien sos?

publicaciones_ra
Hay 10 archivos con la extension establecida dentro de la carpeta.
La base tiene 3359 observaciones.


In [20]:
print("ANALISIS DE LAS BASES\n")

tot = len(df)
mds = (df.id_publicacion == 'MDS').sum()
na = df.id_publicacion.isna().sum()
resto = tot - mds - na

print(  "Resultados con la actualización de David:\n"
        "Total  ",len(df),"\n",
        "MDS    ",mds,"\n",
        "N/A    ",na,"\n",
        "Valid  ",resto, sep=""   
        )

tot = len(df)
mds = (df.id_pub2 == 'MDS').sum()
na = df.id_pub2.isna().sum()
resto = tot - mds - na

print(  "\nResultados anteriores:\n"
        "Total  ",len(df),"\n",
        "MDS    ",mds,"\n",
        "N/A    ",na,"\n",
        "Valid  ",resto, sep=""   
        )

ANALISIS DE LAS BASES

Resultados con la actualización de David:
Total  1050
MDS    0
N/A    88
Valid  962

Resultados anteriores:
Total  1050
MDS    0
N/A    128
Valid  922


In [21]:
# Dropeo MDS - mensajes no validos para el analisis

q_matched = df.id_publicacion.notnull().sum()
q_not_matched = df.id_publicacion.isnull().sum()
q_total = q_matched+q_not_matched
q_unique = len(df[df.id_publicacion.duplicated(keep='first')==False])

print("\n", q_matched, " publicaciones matcheadas\n", 
      q_not_matched, " publicaciones no matcheadas\n",
      q_total, " total de publicaciones\n",
      round(q_matched / q_total * 100,1), "% porcentaje de publicaciones matcheadas\n", 
      q_unique, " publicaciones unicas", sep='')


962 publicaciones matcheadas
88 publicaciones no matcheadas
1050 total de publicaciones
91.6% porcentaje de publicaciones matcheadas
715 publicaciones unicas


In [22]:
df_main.columns

Index(['Unnamed: 0', 'ronda', 'tipo', 'id_publicacion', 'link', 'id', 'nombre',
       'inmobiliaria', 'fecha', 'municipio', 'ciudad', 'pais', 'descripcion',
       'nombre_2', 'precio', 'moneda', 'ambientes', 'habitaciones', 'banos',
       'm2_totales', 'm2_cubiertos', 'cocheras', 'precio_local', 'p_m2',
       'p_m2_above_mdn', 'n_adds', 'n_adds_above_mdn', 'n_adds_muni',
       'n_adds_muni_above_mdn', 'm2_above_mdn', 'common_name', 'capital',
       'random', 'nombre_c', 'precio_c', 'inmobiliaria_c', 'treatment', 'ses',
       'male_name', 'mail', 'male_couple', 'female_couple', 'script',
       'script_string', 'script_string_2', 'randomization_seed',
       'subject_conf', 'from_conf', 'to_conf', 'body_conf', 'date_conf',
       'date_clean_conf', 'date_conf_clean', 'pais_conf', 'match_confiab_conf',
       'match_step_conf', 'id_respuesta', 'subject_inmob', 'from_inmob',
       'to_inmob', 'body_inmob', 'date_inmob', 'cc_inmob', 'thread_id_inmob',
       'mimeType_inmob', 'pais

In [23]:
# Acomodo columnas
df_main = df_main.rename({'pais_conf':'pais'})
df = df.drop(columns=['cantidad_mails','total'])
df = df.rename(columns={'Nom Inmobiliaria':'RA_inmob_name',
                   'Comentario':'RA_match_comment',
                   'fecha':'date_inmob_clean',
                   })

# Construyo columna "tipo"
df['tipo'] = np.where(df['id_publicacion'].isnull(), 'solo_respuesta', 'match_ambas')

# Construyo columna 'match_manual'
df['match_manual'] = 1

## Merge RA con base main

In [24]:
### Reconstruyo id_publicacion de la base RA
# Para ello, tengo que mergear la base publicaciones (que les enviamos a los ra y de donde obtienen la columna id_publicacion) con 
# la base de categorizaciones de RA. De la base publicaciones_ra solo necesito las columnas ['id_publicacion','id_publicacion_hash']
df = publicaciones_ra[['id_publicacion','id_publicacion_hash']].merge(df, on='id_publicacion', indicator=True)
print(df._merge.value_counts())
df = df.drop(columns=['id_publicacion','_merge'])
df = df.rename(columns={'id_publicacion_hash':'id_publicacion'})
df = tools.normaliza_columnas_post_merge(df, suffixes=('_x','_y'))

publicaciones_ra = publicaciones_ra.drop(columns=['id_publicacion'])
publicaciones_ra = publicaciones_ra.rename(columns={'id_publicacion_hash':'id_publicacion'})

both          962
left_only       0
right_only      0
Name: _merge, dtype: int64


In [25]:
vars_publicacion = ['ronda','id_publicacion','link','id','nombre','inmobiliaria','fecha','municipio','ciudad','pais','descripcion','nombre_2','precio','moneda','ambientes','habitaciones','banos','m2_totales','m2_cubiertos','cocheras','precio_local','p_m2','p_m2_above_mdn','n_adds','n_adds_above_mdn','n_adds_muni','n_adds_muni_above_mdn','m2_above_mdn','common_name','capital','random','nombre_c','precio_c','inmobiliaria_c','treatment','ses','male_name','mail','male_couple','female_couple','script','script_string','script_string_2','randomization_seed']
vars_conf_envio  = ['subject_conf','from_conf','to_conf','body_conf','date_conf','date_clean_conf','date_conf_clean','pais_conf','match_confiab_conf','match_step_conf']
vars_rtas_inmob  = ['id_respuesta','subject_inmob','from_inmob','to_inmob','body_inmob','date_inmob','cc_inmob','thread_id_inmob','mimeType_inmob','pais_inmob','date_clean_inmob','date_inmob_clean','match_confiab_inmob','match_step_inmob','match_string_inmob','match2_string_inmob']

In [26]:
### FIXME: ¿Qué pasó con estas 4 observaciones no categorizadas? ¿Dónde las perdimos?
#          ¿Y la publicación?
df_final, test_rtas, test_pubs, new_matched = tools.incorpora_matches_manuales(df_main, 
                                                                        rtas_ra=df, 
                                                                        pubs_ra=publicaciones_ra, 
                                                                        ronda=1, 
                                                                        vars_pubs=vars_publicacion+vars_conf_envio, 
                                                                        vars_rtas=vars_rtas_inmob
                                            ) 
df_final.to_excel(path_datain + fr'\df_final_con_matches_manuales_RA_r1.xlsx')
print(path_datain + fr'\df_final_con_matches_manuales_RA_r1.xlsx', "creado")

Hay respuestas que fueron categorizadas por RA y no están en la base main. Son las siguientes:


,id_respuesta,id_publicacion_y,id_merge,subject_inmob_y,from_inmob_y,to_inmob_y,body_inmob_y,thread_id_inmob_y,pais_y,date_inmob_clean_y,RA,id_pub2,RA_inmob_name,tipo_y,match_manual_y,_merge
2206,842c8a79fca5df0519eb238561c834b144a955adeab881...,c517e5b4c77d58a23a0deb26f7539325697a6e8253f87e...,991.0,DEPTP. AV. ALSINA 1440,jose_lore@loreprop.com.ar,rodriguezmanuel.w@gmail.com,"[<p>BUENOS DIAS, FAVOR LLAMAME A MI CELULAR O ...",18060853dec1581a,arg,2022-04-25 08:39:41,analiariveroborges@gmail.com,NaN,NaN,match_ambas,1.0,right_only
2207,08e7da6c593376904409a439dad29cd643f20e5aab8062...,0f42bab3cc0e0043c19f57783bd245594ed935367a2dd1...,699.0,RESPUESTA GALLO 1455 PISO 4 DEPTO D,estudiorenzi@yahoo.com.ar,rodriguezmanuel.jf@gmail.com,[<p>Buenas tardes:\n\n ...,180480b91480aedc,arg,2022-04-20 14:35:51,analiariveroborges@gmail.com,947,gustavo renzi bienes raices,match_ambas,1.0,right_only
2208,ab7b37e5b900442ee9ccfcef858b14569d339178e74676...,9e2ebdf60f254486d88c35189704ce639f699698d8baa1...,768.0,RE: O'Higgins 700 - | Vicente López | Florida ...,pmasciotra@castanopropiedades.com.ar,phpablogomez@gmail.com,"[<p>Hola Pablo,_x000D_\n_x000D_\n _x000D_\n_x0...",18042d8e44ab69bc,arg,2022-04-19 14:22:24,euge.basile.eb@gmail.com,1626,castano propiedades,match_ambas,1.0,right_only
2209,1b1df40b2a5bffaecaace0c8ae4956203c594e255c26ca...,0a47c875c01fc9dc11ba010f5e9f3210f77d418f395926...,620.0,APARTAMENTO ARRIENDO LA CAROLINA,anavarrete2501@gmail.com,carlos.mz.gonzalez@gmail.com,"[<div class=""WordSection1""><p class=""MsoNormal...",1804ebde37ff9c19,col,2022-04-21 17:07:40,marianomenendez0@gmail.com,4577,anpropiedades,match_ambas,1.0,right_only


Hay publicaciones que fueron enviados a los RA y no están en la base main. Son las siguientes:


,id_publicacion,Unnamed: 0_y,ronda_y,tipo_y,inmobiliaria_y,fecha_y,link_y,municipio_y,ciudad_y,pais_x,descripcion_y,precio_y,moneda_y,habitaciones_y,banos_y,id_y,precio_local_y,mail_y,nombre_backups,to_inmob_y,cantidad_mails,RA,total,Unnamed: 4,_merge
3796,7fab543d06b9163ce4eff1c36730d4a88ebb3690dfe0fc...,1634.0,1.0,match_ambas,asesoria inmobiliaria g & d,2022-03-21,properati.com.pe/detalle/8k3d_alquiler_casa_pu...,lima,lima ampliado,per,se alquila moderno y acogedor departamento tri...,1500.0,usd,3.0,5.0,3047e3d4 4a1d 4d78 91e2 a450643be38b,5602.649902,jdaniel.floresj@gmail.com,departamento en alquiler en pueblo libre,jdaniel.floresj@gmail.com,6.0,silvataniaelizabeth@gmail.com,128.0,5.0,right_only


950
561
C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\para_analisis\df_final_con_matches_manuales_RA_r1.xlsx creado


### Revisiones y otros export

In [27]:
data = test_rtas[test_rtas['_merge']=='right_only'][['from_inmob_y','to_inmob_y']]
for mail_from, mail_to in data.to_records(index=False):
    filtered = df_main[(df_main['from_inmob']==mail_from) & (df_main['to_inmob']==mail_to)]
    if len(filtered)>0:
        print("La observación", df_main[df_main['from_inmob']==mail_from].index, "de la base df_main contiene el mail", mail_from)

    else:
        print("La observación de", mail_from, "no se encuentra en df_main")

La observación de jose_lore@loreprop.com.ar no se encuentra en df_main
La observación de estudiorenzi@yahoo.com.ar no se encuentra en df_main
La observación de pmasciotra@castanopropiedades.com.ar no se encuentra en df_main
La observación de anavarrete2501@gmail.com no se encuentra en df_main


In [28]:
pd.crosstab(df_final['RA'],df_final['tipo'])

tipo,match_ambas
RA,
anaguadalupedepasqua@hotmail.com,115
analiariveroborges@gmail.com,90
canuelobelen@gmail.com,123
castano.fernando.martin@gmail.com,115
euge.basile.eb@gmail.com,69
manuelfmb12@gmail.com,102
marianomenendez0@gmail.com,76
ro.marinsalta@gmail.com,101
rodriffernandez@gmail.com,110


In [29]:
pd.crosstab(df_final.tipo,df_final.ronda)

ronda,1.0,2.0
tipo,,
match_ambas,2165,1281
solo_publicacion,2072,2628
solo_respuesta,95,967


In [30]:
test = df_final.dropna(subset='from_inmob')
test[test['from_inmob'].str.contains('gabriel@interwin.com.ar')]

,Unnamed: 0,ronda,tipo,id_publicacion,link,id,nombre,inmobiliaria,fecha,municipio,ciudad,pais,descripcion,nombre_2,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random,nombre_c,precio_c,inmobiliaria_c,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,subject_conf,from_conf,to_conf,body_conf,date_conf,date_clean_conf,date_conf_clean,pais_conf,match_confiab_conf,match_step_conf,id_respuesta,subject_inmob,from_inmob,to_inmob,body_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais_inmob,date_clean_inmob,date_inmob_clean,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,match_manual,id_merge,RA,Unnamed: 4,id_pub2,RA_inmob_name,RA_match_comment
5634,5637.0,2.0,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,arg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,0f4fc94dfe0e25fca9a83e6d4f05c17665367b3add0a29...,Pablo encontramos esta nueva propiedad para vos,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",f.h.pablogomez@gmail.com,"[<p>Hola Pablo, Hemos encontrado una nueva pro...","Thu, 5 May 2022 08:06:08 -0500",NaN,1809453ee6aac244,multipart/alternative,NaN,2022-05-05,2022-05-05 10:06:08,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5637,5640.0,2.0,solo_respuesta,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,arg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,30bf331b087a74bac24e02fa96859c920c4a98ae26f43a...,ALQUILER ORO TODO RECICLADO A NUEVO,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",f.h.pablogomez@gmail.com,"[<p>ALQUILER ORO TODO RECICLADO A NUEVO , EL ...","Wed, 4 May 2022 13:21:36 -0500",NaN,180904e64528f81f,multipart/alternative,NaN,2022-05-04,2022-05-04 15:21:36,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2831,NaN,1.0,match_ambas,48c84f0d12386f48b36e54e0bc262d83d217d79ff91a51...,properati.com.ar/detalle/4bbvd_alquiler_depart...,2_12_79209587_4244387,departamento en alquiler en oro al 3000,interwin marketing inmobiliario,2022-03-07,caba,buenos aires ampliado,arg,"excelente departamento al frente, 3 ambientes ...",departamento en alquiler en oro al 3000,250000.0,$,3.0,NaN,2.0,120.0,110.0,NaN,250000.0,2083.3333,1.0,17.0,1.0,16.0,1.0,1.0,2.0,1.0,0.5768,oro 3000,250000.0,interwin marketing,hetero,neutral,Manuel,jf.manuel.rodriguez@gmail.com,Juan,Laura,1.0,"Buenos días, mi nombre es Manuel, junto a mi p...",Manuel y Laura.,23.0,Hiciste una consulta por la propiedad Oro Al 3000,Properati <contacts@mailing.properati.com.ar>,jf.manuel.rodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 20:04:26 +0000 (UTC)",2022-04-19,2022-04-19 17:04:26,arg,NaN,NaN,ce2dc7885cac0f79d97f7883e42a994b13c4552348c118...,oro,"""gabriel@interwin.com.ar"" <gabriel@interwin.co...",jf.manuel.rodriguez@gmail.com,[<p>sldos_x000D_\n_x000D_\ngabriel_x000D_\n_x0...,NaN,NaN,1804374b86d3e037,NaN,NaN,NaT,2022-04-19 17:12:43,NaN,NaN,NaN,NaN,1.0,369.0,silvataniaelizabeth@gmail.com,NaN,999,interwin marketing inmobiliario,NaN


In [31]:
# Base para Juli. Solo respuestas + ronda 1.
df_juli = df_final[(df_final['tipo']=='solo_respuesta') & (df_final['ronda']==1)]
df_juli = df_juli.loc[:,'subject_inmob':]
df_juli.to_excel(path_dataout + fr'\mensajes_vacios\no_matcheados_ronda1_v2.xlsx')

In [32]:
import pandas as pd
df_test = pd.read_excel(path_dataout + fr'\bases_operativas\df_final_con_matches_manuales_RA.xlsx')

In [33]:
df_test

,Unnamed: 0,ronda,tipo,id_publicacion,Unnamed: 0.1_conf,inmobiliaria,fecha,link,municipio,ciudad,pais_x,descripcion,precio,moneda,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,id,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,m2_above_mdn,common_name,capital,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,randomization_seed,nombre_2,random,subject_conf,from_conf,to_conf,body_conf,date_conf,pais_y,date_clean_conf,nombre_backups,match_confiab_conf,match_step_conf,nombre_c,precio_c,inmobiliaria_c,Unnamed: 0.1_inmob,Unnamed: 0_inmob,subject_inmob,from_inmob,to_inmob,body_inmob,date_inmob,cc_inmob,thread_id_inmob,mimeType_inmob,pais,date_clean_inmob,match_confiab_inmob,match_step_inmob,match_string_inmob,match2_string_inmob,id_respuesta,id_publicacion.1,date_conf_clean,date_inmob_clean,date_conf_uso_horario,date_inmob_uso_horario,nombre,n_adds_muni,n_adds_muni_above_mdn,_merge,mail_number,time_diff,id_merge,RA,Unnamed: 4,id_pub2,Nom Inmobiliaria,Comentario,to_be_matched,match_manual,match_david
0,0,1,match_ambas,2467.0,907.0,ginevra international realty,2022-04-08,properati.com.ar/detalle/4f4jb_alquiler_depart...,caba,buenos aires ampliado,arg,2 ambientes en venta edificio harbour reside...,1500.0,usd,2.0,NaN,2.0,83.0,72.0,1.0,2_12_79741714_3417277,290250.0,3496.988037,1.0,13.0,1.0,1.0,2.0,1.0,homo,neutral,Pablo,fh.pablogomez@gmail.com,Luis,NaN,2.0,"Hola, somos una pareja interesada en alquilar ...",Pablo y Luis,23.0,NaN,NaN,Hiciste una consulta por la propiedad Lola Mor...,Properati <contacts@mailing.properati.com.ar>,fh.pablogomez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Wed, 20 Apr 2022 14:30:54 +0000 (UTC)",arg,2022-04-20,departamento en alquiler en lola mora al 400,70.0,4.5,lola mora 400,1500.0,ginevra international realty,1976.0,1976.0,Casa en venta con muelle al rio | Villanueva,Ginevra International Realty <info@ginevrasir...,fh.pablogomez@gmail.com,[<p>View Online<https:>\r\n\r\n\r\n\r\n\r\n\r\...,"Sat, 30 Apr 2022 13:32:14 +0000",NaN,1807aac1e523b347,multipart/alternative,arg,2022-04-30,70.0,4.5,"[Match(start=0, end=26, dist=0, matched='ginev...",NaN,2359.0,2467.0,2022-04-20 11:30:54,2022-04-30 10:32:14,NaN,NaN,departamento en alquiler en lola mora al 400,9.0,1.0,both,3.0,14341.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,match_ambas,206.0,2803.0,ginevra international realty,2021-11-03,properati.com.ar/detalle/42w8n_alquiler_depart...,vicente_lopez,buenos aires ampliado,arg,"excelente departamento piso impecable, muy lum...",1200.0,usd,2.0,NaN,1.0,71.0,58.0,NaN,2_12_77369490_3936216,232200.0,3270.422607,1.0,13.0,1.0,1.0,2.0,0.0,homo,neutral,Manuel,jcmanuelrodriguez@gmail.com,Pablo,NaN,1.0,"Buenos días, mi nombre es Manuel, junto a mi p...",Manuel y Pablo.,23.0,NaN,NaN,Hiciste una consulta por la propiedad Corrient...,Properati <contacts@mailing.properati.com.ar>,jcmanuelrodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Tue, 19 Apr 2022 20:18:02 +0000 (UTC)",arg,2022-04-19,departamento en alquiler en corrientes al 300,90.0,3.1,corrientes 300,1200.0,ginevra international realty,632.0,632.0,Casa en venta con muelle al rio | Villanueva,Ginevra International Realty <info@ginevrasir...,jcmanuelrodriguez@gmail.com,"[<p>View Online<https:></https:></p>, <https:>...","Sat, 30 Apr 2022 13:33:54 +0000",NaN,1807aad88f535067,multipart/alternative,arg,2022-04-30,90.0,3.1,"[Match(start=1, end=29, dist=0, matched='ginev...",NaN,2360.0,206.0,2022-04-19 17:18:02,2022-04-30 10:33:54,NaN,NaN,departamento en alquiler en corrientes al 300,4.0,1.0,both,4.0,15435.866667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,match_ambas,206.0,2803.0,ginevra international realty,2021-11-03,properati.com.ar/detalle/42w8n_alquiler_depart...,vicente_lopez,buenos aires ampliado,arg,"excelente departamento piso impecable, muy lum...",1200.0,usd,2.0,NaN,1.0,71.0,58.0,NaN,2_12_77369490_3936216,232200.0,3270.422607,1.0,13.0,1.0,1.0,2.0,0.0,homo,neutral,Ma